In [10]:
# Unet model implementation
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from matplotlib import pyplot as plt

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)
    
#Test code with generated data for architecture testing
"""
def test_a():
    x = torch.randn((3, 1, 256, 256))    
    print(x.dtype)
    print(x.size)
    print("dimenze:" + str(x.ndim))
    print(x.shape)
    
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape 
"""
#Test code with preprocessed data for architecture testing
"""
def test_b():
    PATH_TO_IMAGES = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\train_img\\train2.txt'
    obr = np.loadtxt(PATH_TO_IMAGES, dtype=np.float)
    obr = obr.reshape (1, 1, 256, 256).astype('float32')
    obr = torch.from_numpy(obr)
    print("testb dtype::"+str(obr.dtype))
    print("testb size:"+str(obr.size))
    print("testtb ndim:"+str(obr.ndim))
    print("testtb shape:"+str(obr.shape))      

    model = UNET(in_channels=1, out_channels=1)
    preds = model(obr)

    yz = preds.detach().numpy()
    print()
    print(yz.shape)

    arr = yz
    arr_ = np.squeeze(arr)
    plt.imshow(arr_)
    plt.show()

    
if __name__ == "__main__":
    test_a()
    test_b()
"""

'\ndef test_b():\n    PATH_TO_IMAGES = \'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\train_img\\train2.txt\'\n    obr = np.loadtxt(PATH_TO_IMAGES, dtype=np.float)\n    obr = obr.reshape (1, 1, 256, 256).astype(\'float32\')\n    obr = torch.from_numpy(obr)\n    print("testb dtype::"+str(obr.dtype))\n    print("testb size:"+str(obr.size))\n    print("testtb ndim:"+str(obr.ndim))\n    print("testtb shape:"+str(obr.shape))      \n\n    model = UNET(in_channels=1, out_channels=1)\n    preds = model(obr)\n\n    yz = preds.detach().numpy()\n    print()\n    print(yz.shape)\n\n    arr = yz\n    arr_ = np.squeeze(arr)\n    plt.imshow(arr_)\n    plt.show()\n\n    \nif __name__ == "__main__":\n    test_a()\n    test_b()\n'

In [12]:
#train function #1 (to be tested with my code)
import os
import numpy as np
import torch.nn as nn
import torch.optim as optim
import sys
import import_ipynb 


sys.path.append(os.getcwd())  # path contains python_file.py

import time

def train(model, train_dl, valid_dl, loss_fn, optimizer, acc_fn, epochs=1):
    start = time.time()
    #model.cuda()

    train_loss, valid_loss = [], []

    best_acc = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train(True)  # Set trainind mode = true
                dataloader = train_dl
            else:
                model.train(False)  # Set model to evaluate mode
                dataloader = valid_dl

            running_loss = 0.0
            running_acc = 0.0

            step = 0
            
            # iterate over data
            for x, y in dataloader:
                #x = x.cuda()
                #y = y.cuda()
                step += 1
            
                # forward pass
                if phase == 'train':
                    # zero the gradients
                    optimizer.zero_grad()
                    outputs = model(x)
                    loss = loss_fn(outputs, y)

                    # the backward pass frees the graph memory, so there is no 
                    # need for torch.no_grad in this training pass
                    loss.backward()
                    optimizer.step()
                    # scheduler.step()

                else:
                    with torch.no_grad():
                        outputs = model(x)
                        loss = loss_fn(outputs, y.long())

                # stats - whatever is the phase
                acc = acc_fn(outputs, y)

                running_acc  += acc*dataloader.batch_size
                running_loss += loss*dataloader.batch_size 

                if step % 10 == 0:
                    # clear_output(wait=True)
                    print('Current step: {}  Loss: {}  Acc: {}  AllocMem (Mb): {}'.format(step, loss, acc, torch.cuda.memory_allocated()/1024/1024))
                    # print(torch.cuda.memory_summary())

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_acc / len(dataloader.dataset)

            print('{} Loss: {:.4f} Acc: {}'.format(phase, epoch_loss, epoch_acc))

            train_loss.append(epoch_loss) if phase=='train' else valid_loss.append(epoch_loss)

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    return train_loss, valid_loss    

def acc_metric(predb, yb):
    return (predb.argmax(dim=1) == yb.cuda()).float().mean()


In [13]:
# Loader for data
import os
import numpy as np
#from PIL import Image
from torch.utils.data import Dataset

class MicroscopyDataset (Dataset):
    
    """Loading data fuction"""
    
    def __init__(self, image_dir, mask_dir):
        print("Debug: -here-")
        #print("Path to image:" + str(image_dir))
        #print("Path to mask:" + str(mask_dir))
        self.image_dir = image_dir
        self.mask_dir = mask_dir   
        #list of all files in folder
        self.images = os.listdir(image_dir)
    
    def __len__(self):
        print("Length: -here-")
        #Length of the dataset
        return len(self.images)
    
    def __getitem__(self,i):
        print("Get: -here-")
        #Path for image and mask
        image_path = os.path.join(self.image_dir, self.images[i])
        mask_path = os.path.join(self.mask_dir, self.images[i])
        print(str(image_path))
        print(str(mask_path))
        #load data
        image = np.loadtxt(image_path, dtype = np.float32) 
        mask = np.loadtxt(mask_path, dtype = np.float32)
        image = image.reshape (1, 1, 256, 256).astype('float32') 
        mask = mask.reshape (1, 1, 256, 256).astype('float32')
        #image = torch.from_numpy(image)
        #mask = torch.from_numpy(mask)
        return image, mask

In [14]:
# utility helpers (not all yet used TODO)
import torch
import torchvision
import import_ipynb 
from load_dataset import MicroscopyDataset
from torch.utils.data import DataLoader

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    num_workers=4,
    pin_memory=True,
):
    train_ds = MicroscopyDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = MicroscopyDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x
            y = y
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )
            
    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

In [15]:
#train function #2 (to be tested with my code)
import torch
import torchvision
import import_ipynb 
from load_dataset import MicroscopyDataset
from torch.utils.data import DataLoader


def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())
        
    

In [18]:
#Here is where all the main code should be stored
PATH_TO_IMAGES = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\train_img'
PATH_TO_MASKS = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\train_mask'
PATH_TO_VAL_IMG = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_img'
PATH_TO_VAL_MASK = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_mask'
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 3
NUM_WORKERS = 0
PIN_MEMORY = True
LOAD_MODEL = False

"""
obr = np.loadtxt(PATH_TO_IMAGES, dtype=np.float)
obr = obr.reshape (1, 1, 256, 256).astype('float32')
obr = torch.from_numpy(obr)
print("obr dtype::"+str(obr.dtype))
print("obr size:"+str(obr.size))
print("obr ndim:"+str(obr.ndim))
print("obr shape:"+str(obr.shape))

msk = np.loadtxt(PATH_TO_MASKS, dtype=np.float)
msk = msk.reshape (1, 1, 256, 256).astype('float32')
msk = torch.from_numpy(msk)
print("mask dtype::"+str(msk.dtype))
print("mask size:"+str(msk.size))
print("mask ndim:"+str(msk.ndim))
print("mask shape:"+str(msk.shape))
"""

print("Selected Device: "+ str(DEVICE))
model = UNET(in_channels=1, out_channels=1).to(DEVICE)
obr_loader, msk_loader = get_loaders(
    PATH_TO_IMAGES, 
    PATH_TO_MASKS, 
    PATH_TO_VAL_IMG, 
    PATH_TO_VAL_MASK, 
    BATCH_SIZE,
    NUM_WORKERS,
    PIN_MEMORY,
)


#preds = model(obr)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.01)

"""
detached_preds = preds.detach().numpy()
arr = detached_preds
arr_ = np.squeeze(arr)
plt.imshow(arr_)
plt.show()

detached_mask = msk.detach().numpy()
arr = detached_mask
arr_ = np.squeeze(arr)
plt.imshow(arr_)
plt.show()
"""

#train_loss, valid_loss = train(model, obr, msk, loss_fn, opt, acc_metric, epochs = 1)
if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

    
check_accuracy(msk_loader, model, device=DEVICE)
scaler = torch.cuda.amp.GradScaler()
print("done")


Selected Device: cpu


UnidentifiedImageError: cannot identify image file 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_img\\val1.txt'

In [17]:
#Script for img conversion/rename
from PIL import Image
import os

PATH_TO_IMAGES = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\train_img\\'
PATH_TO_MASKS = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\train_mask\\'
PATH_TO_VAL_IMG = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_img\\'
PATH_TO_VAL_MASK = 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_mask\\'
"""
files = os.listdir(PATH_TO_VAL_IMG)
print(files)
i = 1
for file in files:
    image = np.loadtxt(PATH_TO_VAL_IMG+str(file), dtype = np.float32) 
    
    #print("type:"+str(type(image)))
    #print("ndim:"+str(image.ndim))
    #print("shape:"+str(image.shape))
    #print("size"+str(image.size))
    
    #image = image.reshape (1, 1, 256, 256).astype('float32') 
    
    #im = Image.fromarray(image).convert("L")
    #im.save(PATH_TO_VAL_IMG+"val"+str(i)+".png")
    os.rename(PATH_TO_VAL_IMG+str(file), PATH_TO_VAL_IMG+"val"+str(i)+".txt")
    i = i+1
"""
files = os.listdir(PATH_TO_VAL_MASK)
i = 1
for file in files:
    image = np.loadtxt(PATH_TO_VAL_MASK+str(file), dtype = np.float32) 
    """
    print("type:"+str(type(image)))
    print("ndim:"+str(image.ndim))
    print("shape:"+str(image.shape))
    print("size"+str(image.size))
    """
    #image = image.reshape (1, 1, 256, 256).astype('float32') 
    
    #im = Image.fromarray(image).convert("RGB")
    #im.save(PATH_TO_VAL_MASK+"val"+str(i)+".png")
    os.rename(PATH_TO_VAL_MASK+str(file), PATH_TO_VAL_MASK+"val"+str(i)+".txt")
    i = i+1
    
files = os.listdir(PATH_TO_IMAGES)
i = 1
for file in files:
    image = np.loadtxt(PATH_TO_IMAGES+str(file), dtype = np.float32) 
    """
    print("type:"+str(type(image)))
    print("ndim:"+str(image.ndim))
    print("shape:"+str(image.shape))
    print("size"+str(image.size))
    """
    #image = image.reshape (1, 1, 256, 256).astype('float32') 
    
    #im = Image.fromarray(image).convert("L")
    #im.save(PATH_TO_IMAGES+"train"+str(i)+".png")
    os.rename(PATH_TO_IMAGES+str(file), PATH_TO_IMAGES+"train"+str(i)+".txt")
    i = i+1

files = os.listdir(PATH_TO_MASKS)
i = 1
for file in files:
    image = np.loadtxt(PATH_TO_MASKS+str(file), dtype = np.float32) 
    """
    print("type:"+str(type(image)))
    print("ndim:"+str(image.ndim))
    print("shape:"+str(image.shape))
    print("size"+str(image.size))
    """
    #image = image.reshape (1, 1, 256, 256).astype('float32') 
    
    #im = Image.fromarray(image).convert("L")
    #im.save(PATH_TO_MASKS+"train"+str(i)+".png")
    os.rename(PATH_TO_MASKS+str(file), PATH_TO_MASKS+"train"+str(i)+".txt")
    i = i+1

FileExistsError: [WinError 183] Nelze vytvořit soubor, který již existuje: 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_mask\\val10.txt' -> 'C:\\Users\\Dell\\Desktop\\skola\\rocnik_3\\bakalarka\\implementace\\github_implementace\\CNN_UNet\\val_mask\\val2.txt'